In [25]:
### Creating Combat Log Table

In [26]:
# what needs to happen is that a table needs to be created that stores each turn in combat and the actions that happened in that turn.
# the table should have the following columns:

# create the table in the form of a dataframe
import pandas as pd
import numpy as np
import sqlite3
    
# create the table
combat_log = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Action Result',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])

In [27]:
import Species
import Backgrounds
import Armor_and_Weapons
import Dice_Rolls
import Establishing_Hierarchy
import CHARACTER_CREATOR
import Monsters

import Character_Actions

In [28]:
def generate_situation(character_num):
    # for the number of characters, randomly generate a Player_Character using Random_Character()
    entities = []
    for i in range(character_num):
        entities.append(CHARACTER_CREATOR.Random_Character())

    print('Current HP: ',entities[0].Current_HP)
    # for the number of monsters, ask which monsters to add 

    # for the number of entities, within each entity's flesh out the Active_Conditions and Circumstances dictionaries
    for entity in range(len(entities)):
        other_entities = entities.copy()
        other_entities.pop(entity)
        
        # I wnant to add a key for each other entity in the Active_Conditions dictionaries
        for other_entity in range(len(other_entities)):
            entities[entity].Active_Conditions[str(entities[other_entity].Name)] = []

    return entities

def generate_combat(character_num):
    # create a new combat log
    combat_log_new = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Action Result',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])
    # create situation
    combat_situation = generate_situation(character_num)
    
    # add the Character_Actions.Attack_Action(entity,combat_situation) to each entity's actions
    for entity in range(len(combat_situation)):
        combat_situation[entity].Actions['Attack'] = Character_Actions.Attack_Action(combat_situation[entity],combat_situation,combat_log_new)
        combat_situation[entity].Actions['Dodge'] = Character_Actions.Dodge_Action(combat_situation[entity],combat_situation,combat_log_new)
        
    for entity in range(len(combat_situation)):
        Character_Actions.Equip_Weapon(combat_situation[entity],Character_Actions.Choose_Random_Weapon(combat_situation[entity]))

    # alter the combat log to include the details of every entity in the situation
    # loop through each creature in the situation and add a column for each attribute of the creature
    for creature in range(len(combat_situation)):
        creature_reference = combat_situation[creature]
        combat_log_new[str(creature_reference.Name) + ' Acting True'] = 0
        
        combat_log_new[str(creature_reference.Name) + ' Size'] = creature_reference.Size
        combat_log_new[str(creature_reference.Name) + ' Walking Speed'] = creature_reference.Speed['Walking']
        combat_log_new[str(creature_reference.Name) + ' Flying Speed'] = creature_reference.Speed['Flying']
        combat_log_new[str(creature_reference.Name) + ' Str_Score'] = creature_reference.Str_Score
        combat_log_new[str(creature_reference.Name) + ' Dex_Score'] = creature_reference.Dex_Score
        combat_log_new[str(creature_reference.Name) + ' Con_Score'] = creature_reference.Con_Score
        combat_log_new[str(creature_reference.Name) + ' Int_Score'] = creature_reference.Int_Score
        combat_log_new[str(creature_reference.Name) + ' Wis_Score'] = creature_reference.Wis_Score
        combat_log_new[str(creature_reference.Name) + ' Cha_Score'] = creature_reference.Cha_Score
        combat_log_new[str(creature_reference.Name) + ' Current_HP'] = creature_reference.Current_HP
        combat_log_new[str(creature_reference.Name) + ' Temp_HP'] = creature_reference.Temp_HP
        #combat_log[str(creature_reference.Name) + 'Weapon_Equipped'] = creature_reference.Weapon_Equipped
        #combat_log[str(creature_reference.Name) + 'Armor_Equipped'] = creature_reference.Armor_Equipped
        combat_log_new[str(creature_reference.Name) + ' Active_Conditions'] = creature_reference.Active_Conditions['Self']
        combat_log_new[str(creature_reference.Name) + ' Concentrating'] = creature_reference.Concentrating



    # loop that continues combat until conclusion (aka when one creature's current hp is 0 or less)
        # loop that records each round in 
    combat_round = 0
    combat_log_new['Combat Round'] = combat_round

    # loop that continues combat until conclusion (aka when one creature's current hp is 0 or less)
    while all([combat_situation[entity].Current_HP > 0 for entity in range(len(combat_situation))]):
        combat_round += 1
        combat_log_new['Combat Round'] = combat_round
        combat_log_new = combat_round(combat_situation,combat_log_new)


        for entity in range(len(combat_situation)):
            # randomly choose between the Attack_Action and the Dodge_Action
            action = np.random.choice(['Attack','Dodge'])
            new_round = combat_situation[entity].Actions[action]
            # concat the dict new_round to the combat_log_new
            combat_log_new = pd.concat([combat_log_new, pd.DataFrame(new_round)], ignore_index=True, axis=0)

    # reorganize the columns in combat_log_new 
    # 'Combat Round' goes first, then any column with 'Acting True' goes next, then the Current_HP columns, then the rest of the columns except for Combat Round,
    combat_log_new = combat_log_new.reindex(columns=['Combat Round'] + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Acting True')]) + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Current_HP')]) + list(combat_log_new.columns[1:][~combat_log_new.columns[1:].str.contains('Acting True') & ~combat_log_new.columns[1:].str.contains('Current_HP')]))
    
    return combat_log_new


In [29]:
# remove the number of columns limit
pd.set_option('display.max_columns', None)

In [30]:
generate_combat(4)

Character_Creator - Random Character Function - Class Choice:  <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Create Character Function - Class: <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Create Character Function - Class: <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Create Character Function - Class: <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\

,Combat Round,a1905 Acting True,a8201 Acting True,a4798 Acting True,a8933 Acting True,a5541 Acting True,a3656 Acting True,a1905 Current_HP,a8201 Current_HP,a4798 Current_HP,a8933 Current_HP,a5541 Current_HP,a3656 Current_HP,Action Number,Action Time,Action Name,Action Type,Target,Action Result,Current Allied Ability Check,Current Allied Attack Roll,Current Allied Saving Throw,Current Allied Damage Roll,Current Enemy Ability Check,Current Enemy Attack Roll,Current Enemy Saving Throw,Current Enemy Damage Roll,a1905 Size,a1905 Walking Speed,a1905 Flying Speed,a1905 Str_Score,a1905 Dex_Score,a1905 Con_Score,a1905 Int_Score,a1905 Wis_Score,a1905 Cha_Score,a1905 Temp_HP,a1905 Active_Conditions,a1905 Concentrating,a8201 Size,a8201 Walking Speed,a8201 Flying Speed,a8201 Str_Score,a8201 Dex_Score,a8201 Con_Score,a8201 Int_Score,a8201 Wis_Score,a8201 Cha_Score,a8201 Temp_HP,a8201 Active_Conditions,a8201 Concentrating,a4798 Size,a4798 Walking Speed,a4798 Flying Speed,a4798 Str_Score,a4798 Dex_Score,a4798 Con_Score,a4798 Int_Score,a4798 Wis_Score,a4798 Cha_Score,a4798 Temp_HP,a4798 Active_Conditions,a4798 Concentrating,a8933 Size,a8933 Walking Speed,a8933 Flying Speed,a8933 Str_Score,a8933 Dex_Score,a8933 Con_Score,a8933 Int_Score,a8933 Wis_Score,a8933 Cha_Score,a8933 Temp_HP,a8933 Active_Conditions,a8933 Concentrating,a5541 Size,a5541 Walking Speed,a5541 Flying Speed,a5541 Str_Score,a5541 Dex_Score,a5541 Con_Score,a5541 Int_Score,a5541 Wis_Score,a5541 Cha_Score,a5541 Temp_HP,a5541 Active_Conditions,a5541 Concentrating,a3656 Size,a3656 Walking Speed,a3656 Flying Speed,a3656 Str_Score,a3656 Dex_Score,a3656 Con_Score,a3656 Int_Score,a3656 Wis_Score,a3656 Cha_Score,a3656 Temp_HP,a3656 Active_Conditions,a3656 Concentrating
0,0,1.0,0.0,0.0,0.0,0.0,0.0,10.0,12.0,8.0,12.0,12.0,12.0,Undetermined,Action,Attack,Offense,a4798,Hit,<Dice_Rolls.Ability_Check object at 0x00000232...,<Dice_Rolls.Attack_Roll object at 0x000002320A...,<Dice_Rolls.Saving_Throw object at 0x000002320...,<Dice_Rolls.Damage_Roll object at 0x000002320A...,<Dice_Rolls.Ability_Check object at 0x00000232...,<Dice_Rolls.Attack_Roll object at 0x000002320A...,<Dice_Rolls.Saving_Throw object at 0x000002320...,<Dice_Rolls.Damage_Roll object at 0x000002320A...,Medium,30.0,0.0,14.0,12.0,12.0,8.0,15.0,13.0,0.0,NaN,False,Medium,30.0,0.0,13.0,10.0,14.0,8.0,15.0,12.0,0.0,NaN,False,Medium,30.0,0.0,14.0,10.0,15.0,12.0,13.0,8.0,0.0,NaN,False,Medium,30.0,0.0,11.0,14.0,16.0,9.0,13.0,15.0,0.0,NaN,False,Medium,30.0,0.0,10.0,12.0,17.0,14.0,13.0,8.0,0.0,NaN,False,Medium,30.0,0.0,8.0,14.0,15.0,10.0,12.0,13.0,0.0,NaN,False
1,0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,12.0,8.0,12.0,12.0,12.0,Undetermined,Action,Attack,Offense,a3656,Miss,<Dice_Rolls.Ability_Check object at 0x00000232...,<Dice_Rolls.Attack_Roll object at 0x000002320A...,<Dice_Rolls.Saving_Throw object at 0x000002320...,<Dice_Rolls.Damage_Roll object at 0x000002320A...,<Dice_Rolls.Ability_Check object at 0x00000232...,<Dice_Rolls.Attack_Roll object at 0x000002320A...,<Dice_Rolls.Saving_Throw object at 0x000002320...,<Dice_Rolls.Damage_Roll object at 0x000002320A...,Medium,30.0,0.0,14.0,12.0,12.0,8.0,15.0,13.0,0.0,NaN,False,Medium,30.0,0.0,13.0,10.0,14.0,8.0,15.0,12.0,0.0,NaN,False,Medium,30.0,0.0,14.0,10.0,15.0,12.0,13.0,8.0,0.0,NaN,False,Medium,30.0,0.0,11.0,14.0,16.0,9.0,13.0,15.0,0.0,NaN,False,Medium,30.0,0.0,10.0,12.0,17.0,14.0,13.0,8.0,0.0,NaN,False,Medium,30.0,0.0,8.0,14.0,15.0,10.0,12.0,13.0,0.0,NaN,False
2,0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,12.0,8.0,12.0,12.0,12.0,Undetermined,Action,Attack,Offense,a3656,Miss,<Dice_Rolls.Ability_Check object at 0x00000232...,<Dice_Rolls.Attack_Roll object at 0x000002320A...,<Dice_Rolls.Saving_Throw object at 0x000002320...,<Dice_Rolls.Damage_Roll object at 0x000002320A...,<Dice_Rolls.Ability_Check object at 0x00000232...,<Dice_Rolls.Attack_Roll object at 0x000002320A...,<Dice_Rolls.Saving_Throw object at 0x000002320...,<Dice_Rolls.Damage_Roll object at 0x000002320A...,Medium,30.0,0.0,14.0,12.0,12.0,8.0,15.0,13.0,0.